In [1]:
import andi
import numpy as np
import csv as csv
import numpy as np

from sklearn.metrics import mean_absolute_error
import tensorflow as tf



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras import losses, metrics

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import LSTM

from keras.layers import Dropout
from keras.layers import Bidirectional
from tensorflow.keras.models import load_model
import os
from matplotlib import pyplot as plt
from data_split import data_split

Using TensorFlow backend.


Importing the data

In [2]:
import csv
trajs_from_files = csv.reader(open('challenge_for_scoring/task3.txt','r'), delimiter=';', 
                                        lineterminator='\n',quoting=csv.QUOTE_NONNUMERIC)
validation = [[],[],[]]
for trajs in enumerate(trajs_from_files):
    validation[int(trajs[1][0])-1].append(trajs[1][1:])

Importing the networks, two (three) for inference of exponents and switching times, one for classification of first model, one for classification of second model

In [3]:

model_switch_a_t_new = load_model('nets/task3/1d/task3_new.h5')

model_switch_a_t_diff = load_model('nets/task3/1d/diff_task3.h5')

model_classi_first = load_model('nets/task3/1d/taks_3_classify_frst_checkout200.h5')

model_classi_sec = load_model('nets/task3/1d/taks_3_classify_second_checkout200.h5')

Personalized arctan to convert predictions into switching times stats

In [4]:
def my_atan(x1,x2):
    y=np.arctan2(x1,x2)
    b=y<0
    c=b.astype(int)*(2*np.pi)
    d=y+c 
    return    d;

# 1d case

One of the model works using the positions as inputs

In [9]:
#for 1d
i=200
validation1d=validation[0]
test_tim_step=np.arange(200)
show_time_coll=np.tile(test_tim_step,(len(validation1d),1))
data_show,label_show,traj_show,times_show=data_split(np.asarray(validation1d),
                                                     show_time_coll,
                                                         labels=np.ones(len(validation1d)),
                                                         start_row=0,num_row=len(validation1d),
                                                         traj_len=np.asarray(validation1d).shape[1],n_in=0,n_samples=1,
                                                         p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

pred_a_t_new=model_switch_a_t_new(data_show)


The other models take the increments as inputs

In [10]:
i=200

data_show_diff,label_show,traj_show,times_show=data_split(np.diff(np.asarray(validation1d),axis=1),
                                                     show_time_coll,
                                                         labels=np.ones(len(validation1d)),
                                                         start_row=0,num_row=len(validation1d),
                                                         traj_len=i-1,n_in=0,n_samples=1,
                                                         p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

pred_a_t_diff=model_switch_a_t_diff.predict(data_show_diff)

pred_m1_first=np.argmax(model_classi_first.predict(data_show_diff),axis=1)

pred_m2_sec=np.argmax(model_classi_sec.predict(data_show_diff),axis=1)

The predictions for the switching times are made by taking the average of the 2 models

In [11]:
pr_t_new = my_atan(pred_a_t_new[:,2],pred_a_t_new[:,3])*200/(2*np.pi)

pr_t_diff = my_atan(pred_a_t_diff[:,2],pred_a_t_diff[:,3])*200/(2*np.pi)

pr_t_comb = (pr_t_new+pr_t_diff)/2.

Renaming the classification

In [12]:
pred_m1s=np.copy(pred_m1_first)
pred_m2s=np.copy(pred_m2_sec)

Assembling the total prediction array  in the format required by submission, giving (t,model 1,a1,model2,a2). t is obtained avearging 2 models

In [13]:
predictions1d=np.zeros((len(validation1d),5))
predictions1d[:,0]=pr_t_comb
predictions1d[:,1]=pred_m1s
predictions1d[:,2]=(pred_a_t_new[:,0]+pred_a_t_diff[:,0])/2
predictions1d[:,3]=pred_m2s
predictions1d[:,4]=(pred_a_t_new[:,1]+pred_a_t_diff[:,1])/2

In [14]:
np.save('predictions/task3/task3_predictions1d_final.npy',predictions1d)

In [15]:
pred_to_txt1d = 1*np.ones((len(predictions1d), 6))
for i in range(len(predictions1d)):
    for j in range(5):
        
        pred_to_txt1d[i, j+1] = predictions1d[i][j]
pred_to_txt = pred_to_txt1d
#np.savetxt('task3_tem1d_new.txt', pred_to_txt.astype(float), fmt = '%1.5f', delimiter = ';')

# 2d data

Alternative version, it performed better. For both inference and model detection it is obtained by applying the 1d classifier and averging all dimension independently, we know this may not work well for LW

In [16]:
i=200
for dim in [2]:
    pred_hd_a_t=np.zeros((len(validation[1]),3))
    
    pred_hd_m1_first=np.zeros((len(validation[1]),5))
    
    pred_hd_m2_sec=np.zeros((len(validation[1]),5))
    
    valid_ch_dim=np.asarray(validation[dim-1])    #validation in the chosen dimension
    for jj in range(dim):
        x=valid_ch_dim[:,i*jj:i*(jj+1)]
        test_tim_step=np.arange(i)
        show_time_coll=np.tile(test_tim_step,(len(x),1))
        data_show,label_show,traj_show,times_show=data_split(x,
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1],n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)


        pred_a_t_new=model_switch_a_t_new(data_show)
        pr_t_new = my_atan(pred_a_t_new[:,2],pred_a_t_new[:,3])*200/(2*np.pi)
        pred_hd_a_t[:,:2]+=pred_a_t_new[:,:2]/dim
        pred_hd_a_t[:,2]+=pr_t_new/dim
        
        #model classification uses increments

        
        data_show_diff,label_show,traj_show_diff,times_show=data_split(np.diff(x,axis=1),
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1]-1,n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

        #first model
        pred_m1_first=model_classi_first.predict(data_show_diff)
        pred_hd_m1_first+=pred_m1_first/dim
        
        
        #second model

        pred_m2_sec=model_classi_sec.predict(data_show_diff)
        pred_hd_m2_sec+=pred_m2_sec/dim
            

Assembling the total prediction array  in the format required by submission, giving (t,model 1,a1,model2,a2). For inference, only model new is used and its predictions are averaged in the different dimensions

In [17]:
predictions2d=np.zeros((len(validation[1]),5))
predictions2d[:,0]=pred_hd_a_t[:,2] # t switch
predictions2d[:,1]=np.argmax(pred_hd_m1_first,axis=1)
predictions2d[:,2]=pred_hd_a_t[:,0]  # a1
predictions2d[:,3]=np.argmax(pred_hd_m2_sec,axis=1)
predictions2d[:,4]=pred_hd_a_t[:,1]  #a2

In [18]:
np.save('predictions/task3/task3_predictions2d.npy',predictions2d)

In [19]:
pred_to_txt2d = 2*np.ones((len(predictions2d), 6))
for i in range(len(predictions2d)):
    for j in range(5):
        
        pred_to_txt2d[i, j+1] = predictions2d[i][j]
pred_to_txt = np.concatenate((pred_to_txt1d,pred_to_txt2d))
#np.savetxt('task3_tem2d.txt', pred_to_txt.astype(float), fmt = '%1.5f', delimiter = ';')

# 3d data

Here we again use the alternative version were the 3d data is just an average of the 1d nets

In [ ]:
#for 3d
i=200
for dim in [3]:
    pred_hd_a_t=np.zeros((len(validation[0]),3))
    
    pred_hd_m1_first=np.zeros((len(validation[0]),5))
    pred_hd_m2_sec=np.zeros((len(validation[0]),5))
    valid_ch_dim=np.asarray(validation[dim-1])    #validation in the chosen dimension
    for jj in range(dim):
        x=valid_ch_dim[:,i*jj:i*(jj+1)]
        test_tim_step=np.arange(i)
        show_time_coll=np.tile(test_tim_step,(len(x),1))
        data_show,label_show,traj_show,times_show=data_split(x,
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1],n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

        
        pred_a_t_new=model_switch_a_t_new(data_show)
        pr_t_new = my_atan(pred_a_t_new[:,2],pred_a_t_new[:,3])*200/(2*np.pi)
        pred_hd_a_t[:,:2]+=pred_a_t_new[:,:2]/dim
        pred_hd_a_t[:,2]+=pr_t_new/dim
        
#         #model classification

       
        data_show_diff,label_show,traj_show_diff,times_show=data_split(np.diff(x,axis=1),
                                                             show_time_coll,
                                                                 labels=np.ones(len(x)),
                                                                 start_row=0,num_row=len(x),
                                                                 traj_len=x.shape[1]-1,n_in=0,n_samples=1,
                                                                 p_p=1,hmin=0,hmax=2,limith=False,normalization=True)

        #first model
        pred_m1_first=model_classi_first.predict(data_show_diff)
        pred_hd_m1_first+=pred_m1_first/dim
        
        
        #second model

        pred_m2_sec=model_classi_sec.predict(data_show_diff)
        pred_hd_m2_sec+=pred_m2_sec/dim
                        
        
        
       


Assembling the total prediction array  in the format required by submission, giving (t,model 1,a1,model2,a2). For inference, only model new is used and its predictions are averaged in the different dimensions

In [ ]:
predictions3d=np.zeros((len(validation[0]),5))
predictions3d[:,0]=pred_hd_a_t[:,2]  # t switch
predictions3d[:,1]=np.argmax(pred_hd_m1_first,axis=1)
predictions3d[:,2]=pred_hd_a_t[:,0]  # a1
predictions3d[:,3]=np.argmax(pred_hd_m2_sec,axis=1)
predictions3d[:,4]=pred_hd_a_t[:,1]  #a2    

In [ ]:
np.save('predictions/task3/task3_predictions3d.npy',predictions3d)

In [ ]:
pred_to_txt3d = 3*np.ones((len(predictions3d), 6))
for i in range(len(predictions3d)):
    for j in range(5):
        
        pred_to_txt3d[i, j+1] = predictions3d[i][j]
pred_to_txt_fin = np.concatenate((pred_to_txt,pred_to_txt3d))
np.savetxt('best_sub/task3.txt', pred_to_txt_fin.astype(float), fmt = '%1.5f', delimiter = ';')